## Input the Parameters of the Starlette-SLR GEODYN run:

In [1]:
import pandas as pd
import numpy as np

In [2]:
ll /data/runs_geodyn/iss/results/msis2/msis2_accelon/IIEOUT/


total 346644
-rw-rw-r--. 1 m_geodyn 354960125 Feb 17 22:42 iss190420_2dy.goco05s


In [3]:


########################
# INPUT PARAMETERS:
########################
sat_file = 'iss'
arc = '190420_2dy'
grav_id ='goco05s' 
local_path = '/data/analysis/ISS_GPS_analysis/'
SAT_ID = 9806701
accel_card = 'accelon'


#######################################
# PATH TO DENSITY MODEL RUN of Choice:
#######################################
msis2_model = 'msis2'
path_to_msis2 = '/data/runs_geodyn/'+sat_file+'/results/'+ msis2_model+'/'+ msis2_model+ '_' +accel_card+'/'



### Call the prep-function that loads the data

This may take a few minutes to load.

#### MSIS2

In [4]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# sys.path.insert(0, '/data/analysis/util_funcs/py_geodynreader_gps/')

# from a_ReadISS import ReadISS

# AdjustedParams, Trajectory, Density, Resids = ReadISS(arc, 
#                                                            sat_file,
#                                                            grav_id, 
#                                                            local_path, 
#                                                            path_to_msis2,
#                                                            True)

In [5]:
# WONKY INPUTS:
path_to_data = path_to_msis2
AccelStatus = True


import sys  
sys.path.insert(0, '/data/analysis/notebooks/util_funcs/py_geodynreader_gps/')
SAT_ID = 9806701
sat = sat_file
file_name =  sat + arc + '.'+ grav_id
print('The base file name for this arc is:',file_name,'\n' )
import os

iieout_file  = '/data/analysis/IIEOUT_'+file_name   #path_to_data + 'IIEOUT/'+ file_name


# from b_ReadISS import Save_AdjustedParameters_ISS
# SatMain_AdjustedParams = Save_AdjustedParameters_ISS(SAT_ID, iieout_file, AccelStatus)


The base file name for this arc is: iss190420_2dy.goco05s 



In [28]:

def make_datetime_column_resids(resid_df, YR):
    VERBOSE_timer = False
    if VERBOSE_timer == True:
        import time
        start = time.time()
    else:
        pass

    # resid_df['YYMMDD'] = resid_df['YYMMDD'].astype(int).astype(str)
    # resid_df['HHMM'] =  resid_df['HHMM'].astype(int)
    # resid_df['SEC_UTC'] =  resid_df['SEC_UTC'].astype(float).astype(str)

    resid_df['YYMMDD'] = resid_df['YYMMDD'].astype(int).astype(str)
    resid_df['HHMM'] = resid_df['HHMM'].astype(int).astype(str)


    timeHHMM = [] 
    for i,val in enumerate(resid_df['HHMM']):
    #     print(val)
    #     print(len(val))
        if len(val) == 3:
            timehhmm_val = '0'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 2:
            timehhmm_val = '00'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 1:
            timehhmm_val = '000'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 4:
            timehhmm_val = val
            timeHHMM.append(timehhmm_val)
    #     else:
    #         print('NO WORK')


    #     np.shape(timeHHMM)
    resid_df['timeHHMM'] = timeHHMM

    year  = []
    month = []
    day   = []
    hours  = []
    minutes = []
    secs  = []
    microsecs = []
    for i,val in enumerate(resid_df['YYMMDD']):
    #         print(val[:2])
        if YR < 10:
            year.append('200' + val[:1])
            month.append(val[1:3])
            day.append(val[3:])

            hours.append(resid_df['timeHHMM'][i][:2])
            minutes.append(resid_df['timeHHMM'][i][2:4])

            secs.append(resid_df['SEC_UTC'][i][:2])
            microsecs.append(resid_df['SEC_UTC'][i][3:])
        else:
            year.append('20' + val[:2])
            month.append(val[2:4])
            day.append(val[4:])

            hours.append(resid_df['timeHHMM'][i][:2])
            minutes.append(resid_df['timeHHMM'][i][2:4])

            secs.append(resid_df['SEC_UTC'][i][:2])
            microsecs.append(resid_df['SEC_UTC'][i][3:])

    resid_df['year']  = year
    resid_df['month'] = month
    resid_df['day']   = day
    resid_df['hours']  = hours
    resid_df['minutes'] = minutes
    resid_df['secs']  = secs
    resid_df['microsecs'] = microsecs
    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Loop through and extract indiv date vals:",elapsed)
    else:
        pass

    fix_decimal = []
    for i,val in enumerate(resid_df['secs'].astype(str)):
    #     print(i,val)
        if val.find('.') == 1:
    #             print(i, val)
            fix_decimal.append( '0'+val[:-1])
    #             print(newval)
        else:
            fix_decimal.append( val)

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Fix decimals in the seconds column:",elapsed)
    else:
        pass

    year= list(map(int, resid_df['year'].values))
    month= list(map(int, resid_df['month'].values))
    day= list(map(int, resid_df['day'].values))
    hour= list(map(int, resid_df['hours'].values))
    minute = list(map(int, resid_df['minutes'].values))
    second = list(map(int, fix_decimal))
    microsecond= list(map(int, resid_df['microsecs'].values))

    DATE = list(map(datetime, year,month, day, hour,minute,second,microsecond ))

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
    #         print("Put all dates in a single column:",elapsed)
    else:
        pass


    return(DATE)


def iteration_number(iieout_file):
    '''
    This function opens the iieout file, and returns the final iteration number
    '''
    with open(iieout_file, 'r') as f:
        for line_no, line in enumerate(f):
            if 'CONVERGENCE' in line:
                line_text = line
                # print(line)
    num_iters = float(line_text[39:42])-1
    return num_iters

def read_observed_resids_gps(iieout_file, Year = 19, VERBOSE_timer = False):
    '''
    This function reads in the residuals from the massive IIEOUT file.


    For residuals, there are specific station-satellite configurations.  
    It is prudent to read in each configuration and save which satellites make it up.  
    This is much harder to do than simply reading in all resuiduals (as I did before)


    '''





    if VERBOSE_timer == True:
        import time
        start = time.time()
    else:
        pass

    #############################
    # - BEGIN RESIDUAL READER -
    #############################


    # How many iterations are in this run?
    iteration = str(int(iteration_number(iieout_file)))
    VERBOSE_timer = True
    #
    # ----------------------------------------------------------------------------------
    #
    # We first find how many observations there are. 
    # We use the observation number to find where to STOP reading in data. 
    # We find this by finding the Summary by Measurment Header
    text_smry_meas = 'RESIDUAL SUMMARY BY MEASUREMENT TYPE FOR ARC  1 INNER ITERATION  '+ (iteration) +' OF GLOBAL ITERATION 1'
    #
    # Loop through the iieout file and find where the above header exists (it should only show up once)
    # For ISS-GPS runs, there are four TYPES of residuals.  We would like to read these in automatically
    #        PCE X
    #        PCE X
    #        PCE X
    #        DSS1WRNG 
    #
    with open(iieout_file, 'r') as f:
        for line_no, line_text in enumerate(f):
            if text_smry_meas in line_text:
                # Save the exact number where this section (based on the header) appears:
                text_smry_meas_line_no = line_no
    #
    # Save out the Observation residual types:
    # I read in the whole section and stop reading in when the first value of the line is no longer 0
    count_lines = 1
    line = linecache.getline(iieout_file,text_smry_meas_line_no+count_lines)
    if int(line[0]) == 1: 
        is_integer = True
    else:
        print('Started out as wrong value in SUMMRY of MEASURMENTS')
        is_integer = False

    while is_integer == True:
        try:
            int(line[0])
    #         print('Its an INTEGER')
            line = linecache.getline(iieout_file,text_smry_meas_line_no+count_lines)
            count_lines += 1
            is_integer = True
        except:
    #         print(line[0])
    #         print('not an INTEGER')
            is_integer = False
            count_lines -= 3
    # print(count_lines)
    # print(linecache.getline(iieout_file,text_smry_meas_line_no+count_lines))
    resid_meas_summry = pd.read_csv(iieout_file, 
                               skiprows = text_smry_meas_line_no + 2  , 
                               nrows =  count_lines-2,
                               sep = '\s+',
                               names =['Binary' ,
                                        'NUMBER' ,
                                        'MEAN' ,
                                        'RMS' ,
                                        'No.-WTD' ,
                                        'WTD-MEAN' ,
                                        'WTD-RMS' ,
                                        'TYPE1' ,
                                        'TYPE2' ,
                                        ] ,
                                        )
    # Fix some formatting issues with the way the TYPES were read in
    # concatenate the PCE and X (Y, Z) to make one column
    type_fixed = []
    for i,val in resid_meas_summry.iterrows():
        try:
            float(val['TYPE2'])
            type_fixed.append(val['TYPE1'])
            pass
        except:
            type_fixed.append(val['TYPE1'] + val['TYPE2'])
    resid_meas_summry['TYPE'] = type_fixed          

    #
    # ----------------------------------------------------------------------------------
    #
    '''
    Now find all the instances of the OBSERVATION RESIDUALS 
    header at the last iteration.  

    We will want to store these into a dictionary and save out:
        - configuration type
        - contributing satellites

        It will probably be best to input data base on BLOCK number

    '''
    text_obs_resid = 'OBSERVATION RESIDUALS FOR ARC  1 FOR INNER ITERATION  '+ (iteration)
    end_of_section = 'RESIDUAL SUMMARY BY STATION AND TYPE FOR ARC  1 INNER ITERATION  '+ (iteration)
    lines_list_1 = [] 
    lines_list_2 = []

    # The below grabs the line numbers of the section headers 
    # The Observation Residuals end at the first instance of the Summary by Station
    with open(iieout_file, 'r') as f:
        for line_no, line in enumerate(f):
            if text_obs_resid in line:
                lines_list_1.append(line_no)
            elif end_of_section in line:
                lines_list_2.append(line_no)

    residual_range  = np.arange(lines_list_1[0], lines_list_2[0]+1)
    # block_num_counter = 0

    list_config_type  = []
    list_SAT_main     = []
    list_note         = []
    list_SAT_1        = []
    list_SAT_2        = []
    list_YYMMDD       = []
    list_HHMM         = []
    list_SEC_UTC      = []
    list_Observation  = []
    list_Residual     = []
    list_RatiotoSigma = []
    list_Elev1        = []
    list_Elev2        = []
    list_OBS_No       = []
    list_Block        = []

    for i,val in enumerate(residual_range):
        line = linecache.getline(iieout_file,val)
        if 'STATION-SATELLITE CONFIGURATION' in line:
    #         print('HEADER Type 1')
            config_type = line[35:44]
            SAT_main = line[54:62]
            SAT_1 = line[72:80]
            SAT_2 = line[90:98]
            note = np.nan
            # print(Stat_Sat_Config[35:43])
            # print(Stat_Sat_Config[54:62])
            # print(Stat_Sat_Config[72:80])
            # print(Stat_Sat_Config[90:98])
        elif 'STATION-SAT CONFIG.' in line:
            if 'DSS1WRNG' in line:
    #             print('HEADER Type 2-1')
                config_type = line[46:56]
                SAT_main = line[65:73]
                note = np.nan
                SAT_1 = line[83:91]
                SAT_2 = line[100:109]
            else:
    #             print('HEADER Type 2-2')
                config_type = line[46:56]
                SAT_main = np.nan
                note = line[55:63]
                SAT_1 = line[65:74]
                SAT_2 = np.nan

        try:
            BLOCK_no = int(line[117:125])
            YYMMDD       = line[1:8]
            HHMM         = line[8:13]
            SEC_UTC      = line[13:23]
            Observation  = line[26:43]
            Residual     = line[46:58]
            RatiotoSigma = line[60:70]
            Elev1        = line[71:84]
            Elev2        = line[85:96]
            OBS_No       = line[106:117]
            Block        = line[117:125]

            list_config_type.append(config_type)
            list_SAT_main.append(SAT_main)
            list_note.append(note)
            list_SAT_1.append(SAT_1)
            list_SAT_2.append(SAT_2)
            list_YYMMDD.append(YYMMDD)
            list_HHMM.append(HHMM)
            list_SEC_UTC.append(SEC_UTC)
            list_Observation.append(Observation)
            list_Residual.append(Residual)
            list_RatiotoSigma.append(RatiotoSigma)
            list_Elev1.append(Elev1)
            list_Elev2.append(Elev2)
            list_OBS_No.append(OBS_No)
            list_Block.append(Block)
        except:
    #         print('Not a data block', line[117:125]) 
            pass

    resids_dict= {'StatSatConfig' : list_config_type,
                  'Sat_main'      : list_SAT_main   ,
                  'SAT_1'         : list_SAT_1      ,
                  'SAT_2'         : list_SAT_2      ,
                  'Note'          : list_note       ,
                  'YYMMDD'        : list_YYMMDD      ,
                  'HHMM'          : list_HHMM        ,
                  'SEC_UTC'       : list_SEC_UTC      ,
                  'Observation'   : list_Observation  ,
                  'Residual'      : list_Residual     ,
                  'RatiotoSigma'  : list_RatiotoSigma ,
                  'Elev1'         : list_Elev1       ,
                  'Elev2'         : list_Elev2       ,
                  'OBS_No'        : list_OBS_No      ,
                  'Block'         : list_Block       ,
                 } 

    resids_df = pd.DataFrame.from_dict(resids_dict)
    linecache.clearcache()
    #
    # ----------------------------------------------------------------------------------
    #
    # Fix the date column
    # Year = 19
    dates = make_datetime_column_resids(resids_df, Year)
    resids_df['Date'] = dates

    # The ratio-to-sigma columns has some weird strings in it
    #        ValueError: could not convert string to float: ' -16.0620*'
    #        remove them
    fix_string = []
    for i,val in enumerate(resids_df['RatiotoSigma']):
        try:
            float(val)
            fix_string.append(val)
        except:
            # print(i, val)
            fix_string.append(val[:-1])

    resids_df['RatiotoSigma'] = fix_string
#   
# Some of the elevations are empty.  Replace the empty strings with nans
#
    elev_fix = []
    for i,val in enumerate(A_FILE['Elev1']):
        try:
            float(val)
            elev_fix.append(float(val))
        except:
            elev_fix.append(np.nan)
    resids_df['Elev1'] = elev_fix
    elev_fix = []
    for i,val in enumerate(A_FILE['Elev2']):
        try:
            float(val)
            elev_fix.append(float(val))
        except:
            elev_fix.append(np.nan)
    resids_df['Elev2'] = elev_fix
        
    resids_df['Observation']  = resids_df['Observation'].astype(float)
    resids_df['Residual']     = resids_df['Residual'].astype(float)
    resids_df['RatiotoSigma'] = resids_df['RatiotoSigma'].astype(float)


    return(resids_df)




In [29]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import os.path
import linecache
# VERBOSE_timer = True


In [30]:
A_FILE = read_observed_resids_gps(iieout_file, Year=19)

In [31]:
A_FILE

,StatSatConfig,Sat_main,SAT_1,SAT_2,Note,YYMMDD,HHMM,SEC_UTC,Observation,Residual,...,Block,timeHHMM,year,month,day,hours,minutes,secs,microsecs,Date
0,DSS1WRNG,9806701,5044284,5553175,NaN,190420,2118,0.000000,209.464035,-0.075906,...,1,2118,2019,04,20,21,18,0,000000,2019-04-20 21:18:00
1,DSS1WRNG,9806701,5044284,5553175,NaN,190420,2118,1.000000,206.595126,-0.072653,...,1,2118,2019,04,20,21,18,1,000000,2019-04-20 21:18:01
2,DSS1WRNG,9806701,5044284,5553175,NaN,190420,2118,2.000000,203.725479,-0.068320,...,1,2118,2019,04,20,21,18,2,000000,2019-04-20 21:18:02
3,DSS1WRNG,9806701,5044284,5553175,NaN,190420,2118,3.000000,200.855102,-0.065772,...,1,2118,2019,04,20,21,18,3,000000,2019-04-20 21:18:03
4,DSS1WRNG,9806701,5044284,5553175,NaN,190420,2118,4.000000,197.984004,-0.060666,...,1,2118,2019,04,20,21,18,4,000000,2019-04-20 21:18:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301329,DSS1WRNG,9806701,5345214,5652315,NaN,190422,239,40.000000,1596.112520,0.068724,...,18542,0239,2019,04,22,02,39,40,000000,2019-04-22 02:39:40
301330,DSS1WRNG,9806701,5345214,5652315,NaN,190422,239,41.000000,1590.391982,0.072347,...,18542,0239,2019,04,22,02,39,41,000000,2019-04-22 02:39:41
301331,DSS1WRNG,9806701,5345214,5652315,NaN,190422,239,42.000000,1584.675257,0.071817,...,18542,0239,2019,04,22,02,39,42,000000,2019-04-22 02:39:42
301332,DSS1WRNG,9806701,5345214,5652315,NaN,190422,239,43.000000,1578.962361,0.067737,...,18542,0239,2019,04,22,02,39,43,000000,2019-04-22 02:39:43


In [ ]:
# dates = make_datetime_column_resids(resids_df)


# D['Date'] = dates


# fix_string = []
# for i,val in enumerate(D['Ratio to sigma']):
#     try:
#         float(val)
#         fix_string.append(val)
#     except:
#         # print(i, val)
#         fix_string.append(val[:-1])

# D['Ratio_to_sigma_fixed'] = fix_string
#     return D

In [ ]:
#
'''
Now find all the instances of the OBSERVATION RESIDUALS 
header at the last iteration.  

We will want to store these into a dictionary and save out:
    - configuration type
    - contributing satellites
    
    It will probably be best to input data base on BLOCK number
    
    
'''
# text_obs_resid = 'OBSERVATION RESIDUALS FOR ARC  1 FOR INNER ITERATION  '+ (iteration)
# end_of_section = 'RESIDUAL SUMMARY BY STATION AND TYPE FOR ARC  1 INNER ITERATION  '+ (iteration)
# lines_list_1 = [] #np.empty(np.size(num_observations))
# lines_list_2 = [] #np.empty(np.size(num_observations))

# # The below grabs the line numbers of the section headers
# with open(iieout_file, 'r') as f:
#     for line_no, line in enumerate(f):
#         if text_obs_resid in line:
# #                 print(line_no)
#             lines_list_1.append(line_no)
# #         line_no_1 =lines_list 
#         elif end_of_section in line:
#             lines_list_2.append(line_no)


# # lines = search_iiesout_all_line_numbers(iieout_file, text)
# line_no_1 = lines_list_1[0]
# line_no_2 = lines_list_2[-1]
# #-------------------------------------------------------------------------------
# #-------------------------------------------------------------------------------
# '''
# Use the first and last line numbers from above to select
# the sections of data that contains the observation residuals.
# The outputted csv data are stored as A
# '''
# RESID_OBSERV = pd.read_csv(iieout_file, 
#                            skiprows = line_no_1 + 6 , 
#                            nrows =  int((line_no_2 - line_no_1) ),
#                            sep = '\s+',
#                            names = ['YYMMDD'        ,
#                                     'HHMM'          ,
#                                     'Sec-UTC-R'     ,
#                                     'Observation'   ,
#                                     'Residual'      ,
#                                     'Ratio to sigma',
#                                     'Elev1'         ,
#                                     'Elev2'         ,
#                                     'OBS No.'       ,
#                                     'Block']        ,
#                                     )



# if VERBOSE_timer == True:
#     end = time.time()
#     elapsed = end - start
#     print("Elapsed time after line search setup:",elapsed)
# else:
#     pass
# #-------------------------------------------------------------------------------
# #-------------------------------------------------------------------------------
# ''' 
#   We now need to fix the data that contains a lot of misplaced
#   characters, random strings, and headers
# '''
# A = pd.DataFrame(RESID_OBSERV)  # input
# index_list = []
# for index, row in A.iterrows():
#     try:
#         float(row['OBS No.'])
#         float(row['HHMM'])
#     except:
#         index_list.append(index)
#         continue

# B=A.drop(index_list)
# if VERBOSE_timer == True:
#     end = time.time()
#     elapsed = end - start
#     print("Elapsed time after loop through then drop indecies:",elapsed)
# else:
#     pass

# #-------------------------------------------------------------------------------
# #-------------------------------------------------------------------------------
# ''' 
#   We locate the index of the last observation number and remove 
#   all datapoints that are after it in the DataFrame
# '''
# C = B.reset_index()
# index_drops = np.arange(C[C['OBS No.']==str(int(num_observations))].index[0]+1,  C.index.stop)
# index_drops

# D = C.drop(index_drops)
# if VERBOSE_timer == True:
#     end = time.time()
#     elapsed = end - start
#     print("Elapsed time after dropping all bad indicies after last obs no.:",elapsed)

# # dates = make_datetime_column_resids(D)
# # D['Date'] = dates

# # fix_string = []
# # for i,val in enumerate(D['Ratio to sigma']):
# #     try:
# #         float(val)
# #         fix_string.append(val)
# #     except:
# #         # print(i, val)
# #         fix_string.append(val[:-1])

# # D['Ratio_to_sigma_fixed'] = fix_string
# #     return D

